In [1]:
import os
import re
import nltk
import shutil
from nltk.corpus import stopwords
from zipfile import ZipFile, BadZipFile

# Télécharger les stopwords une fois
nltk.download('stopwords')

# Obtenir les stopwords en français et anglais
stop_words_fr = set(stopwords.words('french'))
stop_words_en = set(stopwords.words('english'))

def clean_srt(input_file, output_file, encoding='utf-8'):
    try:
        with open(input_file, 'r', encoding=encoding) as file:
            lines = file.readlines()
        
        cleaned_lines = []
        
        for line in lines:
            if not line.strip().isdigit() and '-->' not in line:
                words = line.strip().split()
                filtered_words = [word for word in words if word.lower() not in stop_words_fr and word.lower() not in stop_words_en]
                cleaned_lines.append(' '.join(filtered_words))
        
        with open(output_file, 'w', encoding=encoding) as file:
            file.write('\n'.join(cleaned_lines))
    except UnicodeDecodeError:
        print(f"Erreur de décodage avec l'encodage {encoding}. Essayez un autre encodage.")

def extract_archive(archive_path, extract_to):
    if archive_path.endswith('.zip'):
        try:
            with ZipFile(archive_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
        except BadZipFile:
            print(f"Erreur: {archive_path} n'est pas un fichier zip fonctionnel.")
    # Ajouter d'autres formats d'archive si nécessaire

def delete_srt_files_and_empty_dirs(root_folder):
    # Parcourir tous les fichiers et dossiers à partir du dossier racine
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.srt') or file.endswith('.zip'):
                try:
                    os.remove(file_path)  # Supprimer le fichier .srt ou .zip
                except PermissionError:
                    print(f"Erreur de permission: Impossible de supprimer {file_path}")

    # Supprimer les dossiers vides
    for subdir, dirs, _ in os.walk(root_folder, topdown=False):
        for dir in dirs:
            dir_path = os.path.join(subdir, dir)
            try:
                os.rmdir(dir_path)  # Supprimer le dossier s'il est vide
            except OSError:
                pass  # Ignorer l'erreur si le dossier n'est pas vide

def process_all_srt_files(root_folder, encoding='utf-8'):
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.srt'):
                input_file = file_path
                output_file = os.path.join(subdir, file.replace('.srt', '_cleaned.txt'))
                clean_srt(input_file, output_file, encoding)
            elif file.endswith('.zip'):
                extract_to = os.path.join(subdir, file.replace('.zip', ''))
                os.makedirs(extract_to, exist_ok=True)
                extract_archive(file_path, extract_to)
                process_all_srt_files(extract_to, encoding)

    # Supprimer les fichiers .srt, .zip et les dossiers vides après traitement
    delete_srt_files_and_empty_dirs(root_folder)

# Utilisation de la fonction pour traiter tous les fichiers .srt dans le dossier racine
root_folder = './subsFused'  # Remplacez par le chemin de votre dossier racine
process_all_srt_files(root_folder, encoding='latin1')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Erreur: ./subsFused\bionicwoman\bionicwomans01VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\bionicwoman\bionicwomans01VO.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\blade\blades01VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\entourage\entouragesallVF\Entourage.502.VO.VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\entourage\entouragesallVO\Entourage.502.VO.VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\prisonbreak\prisonbreaks01VO.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\robinhood\robinhoods01s02VO\Robin.Hood.S1.DVDRip.VO.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\xfiles\xfiless01VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subsFused\xfiles\xfiless03VF.zip n'est pas un fichier zip fonctionnel.


In [8]:
import os

def fusionner_txt_dans_sous_dossier(dossier_racine):
    """
    Fusionne tous les fichiers .txt d'une série, y compris ceux dans les sous-dossiers,
    dans un seul fichier dans chaque sous-dossier de série, puis supprime les fichiers originaux.
    
    Args:
    dossier_racine (str): Chemin du dossier racine contenant les sous-dossiers par série.
    """
    # Parcourt chaque sous-dossier dans le dossier racine (chaque série a son propre sous-dossier)
    for serie in os.listdir(dossier_racine):
        chemin_serie = os.path.join(dossier_racine, serie)
        
        # Vérifie si c'est un dossier (car on ne veut traiter que les sous-dossiers)
        if os.path.isdir(chemin_serie):
            contenu_fusionne = ""
            fichiers_a_supprimer = []
            
            # Parcourt tous les sous-dossiers et fichiers .txt dans le sous-dossier de la série
            for racine, sous_dossiers, fichiers in os.walk(chemin_serie):
                # Filtre les fichiers .txt dans chaque dossier
                fichiers_txt = [f for f in fichiers if f.endswith('.txt')]
                
                # Lit et fusionne le contenu de chaque fichier .txt
                for fichier_txt in fichiers_txt:
                    chemin_fichier = os.path.join(racine, fichier_txt)
                    with open(chemin_fichier, 'r', encoding='latin1') as f_entree:
                        contenu_fusionne += f_entree.read() + '\n'
                    fichiers_a_supprimer.append(chemin_fichier)

            # Écrit le fichier fusionné dans le sous-dossier de la série
            if contenu_fusionne:  # Si du contenu a été trouvé
                fichier_sortie = os.path.join(chemin_serie, f'{serie}.txt')
                with open(fichier_sortie, 'w', encoding='utf-8') as f_sortie:
                    f_sortie.write(contenu_fusionne)
                print(f'Fichier fusionné pour la série "{serie}" créé: {fichier_sortie}')
                
                # Supprime les fichiers originaux
                for fichier in fichiers_a_supprimer:
                    os.remove(fichier)
                print(f'Fichiers originaux pour la série "{serie}" supprimés.')
            else:
                print(f'Pas de fichiers .txt trouvés pour la série "{serie}".')

# Exemple d'utilisation
dossier_racine = './subsFused'  # Remplacez par le chemin réel de votre dossier racine

fusionner_txt_dans_sous_dossier(dossier_racine)

Fichier fusionné pour la série "24" créé: ./subsFused\24\24.txt
Fichiers originaux pour la série "24" supprimés.
Fichier fusionné pour la série "90210" créé: ./subsFused\90210\90210.txt
Fichiers originaux pour la série "90210" supprimés.
Fichier fusionné pour la série "alias" créé: ./subsFused\alias\alias.txt
Fichiers originaux pour la série "alias" supprimés.
Fichier fusionné pour la série "angel" créé: ./subsFused\angel\angel.txt
Fichiers originaux pour la série "angel" supprimés.
Fichier fusionné pour la série "battlestargalactica" créé: ./subsFused\battlestargalactica\battlestargalactica.txt
Fichiers originaux pour la série "battlestargalactica" supprimés.
Fichier fusionné pour la série "betteroffted" créé: ./subsFused\betteroffted\betteroffted.txt
Fichiers originaux pour la série "betteroffted" supprimés.
Fichier fusionné pour la série "bionicwoman" créé: ./subsFused\bionicwoman\bionicwoman.txt
Fichiers originaux pour la série "bionicwoman" supprimés.
Fichier fusionné pour la séri